In [4]:
# Import libraries
import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import wordnet
import spacy
from textblob import TextBlob
import pandas as pd

In [5]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [6]:
# Define the word set
words = ["unbelievably", "happiest", "disagreement", "rewritten", "wolves", "singing", "unfolded"]
words

['unbelievably',
 'happiest',
 'disagreement',
 'rewritten',
 'wolves',
 'singing',
 'unfolded']

## Task 1: Stemming and Lemmatization Results

Using multiple libraries and algorithms

In [7]:
# Initialize stemmers
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer('english')

In [8]:
def porter_stemming(word_list):
    """Apply Porter Stemmer to words"""
    return [porter_stemmer.stem(word) for word in word_list]

In [9]:
def snowball_stemming(word_list):
    """Apply Snowball Stemmer to words"""
    return [snowball_stemmer.stem(word) for word in word_list]

In [10]:
def spacy_lemmatization(word_list):
    """Apply spaCy lemmatization to words"""
    lemmas = []
    for word in word_list:
        doc = nlp(word)
        lemmas.append(doc[0].lemma_)
    return lemmas

In [11]:
def textblob_lemmatization(word_list):
    """Apply TextBlob lemmatization to words"""
    lemmas = []
    for word in word_list:
        blob = TextBlob(word)
        lemmas.append(blob.words[0].lemmatize())
    return lemmas

In [12]:
# Apply all stemming and lemmatization methods
porter_results = porter_stemming(words)
snowball_results = snowball_stemming(words)
spacy_results = spacy_lemmatization(words)
textblob_results = textblob_lemmatization(words)

In [13]:
# Create results dataframe
results_df = pd.DataFrame({
    'Original Word': words,
    'Porter Stemmer': porter_results,
    'Snowball Stemmer': snowball_results,
    'spaCy Lemmatizer': spacy_results,
    'TextBlob Lemmatizer': textblob_results
})

results_df

,Original Word,Porter Stemmer,Snowball Stemmer,spaCy Lemmatizer,TextBlob Lemmatizer
0,unbelievably,unbeliev,unbeliev,unbelievably,unbelievably
1,happiest,happiest,happiest,happy,happiest
2,disagreement,disagr,disagr,disagreement,disagreement
3,rewritten,rewritten,rewritten,rewrite,rewritten
4,wolves,wolv,wolv,wolf,wolf
5,singing,sing,sing,singe,singing
6,unfolded,unfold,unfold,unfold,unfolded


## Task 2: Morpheme Segmentation (Prefixes, Roots, Suffixes)

In [14]:
# Define morpheme database
morpheme_data = {
    'prefixes': {
        'un': 'not/reverse',
        're': 'again',
        'dis': 'opposite',
        'in': 'not',
        'im': 'not',
        'ir': 'not',
        'il': 'not',
        'pre': 'before',
        'sub': 'under',
        'super': 'above',
        'trans': 'across'
    },
    'suffixes': {
        'ly': 'adverb',
        'ness': 'noun (quality)',
        'ment': 'noun (action/result)',
        'ing': 'gerund/present participle',
        'ed': 'past tense/adjective',
        'er': 'agent noun/comparative',
        'est': 'superlative',
        'tion': 'noun (action)',
        'sion': 'noun (action)',
        'able': 'adjective (capable)',
        'ible': 'adjective (capable)',
        'ful': 'adjective (full of)',
        'less': 'adjective (without)',
        's': 'plural/3rd person',
        'es': 'plural/3rd person',
        'ize': 'verb (make)',
        'ise': 'verb (make)',
        'ous': 'adjective',
        'ious': 'adjective',
        'al': 'adjective',
        'en': 'verb/adjective'
    },
    'common_roots': {
        'believe': 'believe',
        'happy': 'happy',
        'agree': 'agree',
        'write': 'write',
        'wolf': 'wolf',
        'sing': 'sing',
        'fold': 'fold',
        'bel': 'beautiful/well',
        'sent': 'feel',
        'graph': 'write',
        'port': 'carry',
        'duc': 'lead',
        'ject': 'throw',
        'struct': 'build'
    }
}

In [15]:
def extract_morphemes(word):
    """
    Extract morphemes from a word.
    Returns: {prefixes: [], root: '', suffixes: []}
    """
    word_lower = word.lower()
    prefixes = []
    suffixes = []
    root = word_lower
    
    # Extract prefixes (check from longest to shortest)
    prefix_list = sorted(morpheme_data['prefixes'].keys(), key=len, reverse=True)
    for prefix in prefix_list:
        if word_lower.startswith(prefix) and len(word_lower) > len(prefix):
            prefixes.append(prefix)
            root = word_lower[len(prefix):]
            word_lower = root
            break
    
    # Extract suffixes (check from longest to shortest)
    suffix_list = sorted(morpheme_data['suffixes'].keys(), key=len, reverse=True)
    for suffix in suffix_list:
        if root.endswith(suffix) and len(root) > len(suffix):
            suffixes.insert(0, suffix)
            root = root[:-len(suffix)]
            break
    
    return {
        'prefixes': prefixes,
        'root': root,
        'suffixes': suffixes
    }

In [16]:
def format_morpheme_output(word):
    """Format morpheme analysis for display"""
    morphemes = extract_morphemes(word)
    
    prefix_str = ' + '.join(morphemes['prefixes']) if morphemes['prefixes'] else '(none)'
    root_str = morphemes['root']
    suffix_str = ' + '.join(morphemes['suffixes']) if morphemes['suffixes'] else '(none)'
    
    segmentation = f"{prefix_str} | {root_str} | {suffix_str}"
    
    prefix_meanings = [morpheme_data['prefixes'].get(p, 'unknown') for p in morphemes['prefixes']]
    suffix_meanings = [morpheme_data['suffixes'].get(s, 'unknown') for s in morphemes['suffixes']]
    root_meaning = morpheme_data['common_roots'].get(morphemes['root'], 'root')
    
    meanings = []
    if prefix_meanings and morphemes['prefixes'][0]:
        meanings.append(f"Prefix: {', '.join(prefix_meanings)}")
    meanings.append(f"Root: {root_meaning}")
    if suffix_meanings and morphemes['suffixes']:
        meanings.append(f"Suffix: {', '.join(suffix_meanings)}")
    
    return {
        'word': word,
        'segmentation': segmentation,
        'meanings': ' | '.join(meanings)
    }

In [17]:
# Extract and display morphemes for all words
morpheme_results = [format_morpheme_output(word) for word in words]
morpheme_df = pd.DataFrame(morpheme_results)
morpheme_df

,word,segmentation,meanings
0,unbelievably,un | believab | ly,Prefix: not/reverse | Root: root | Suffix: adverb
1,happiest,(none) | happi | est,Root: root | Suffix: superlative
2,disagreement,dis | agree | ment,Prefix: opposite | Root: agree | Suffix: noun ...
3,rewritten,re | writt | en,Prefix: again | Root: root | Suffix: verb/adje...
4,wolves,(none) | wolv | es,Root: root | Suffix: plural/3rd person
5,singing,(none) | sing | ing,Root: sing | Suffix: gerund/present participle
6,unfolded,un | fold | ed,Prefix: not/reverse | Root: fold | Suffix: pas...


## Task 3: Classification of Morphological Changes (Inflectional vs Derivational)

In [18]:
# Define morphological change types
inflectional_morphemes = {
    'ed': 'past tense marker (does not change word class)',
    's': 'plural/3rd person singular (does not change word class)',
    'es': 'plural/3rd person singular (does not change word class)',
    'ing': 'present participle (limited context)',
    'er': 'comparative (limited to adjectives/adverbs)',
    'est': 'superlative (limited to adjectives/adverbs)'
}

derivational_morphemes = {
    'ly': 'changes adjective to adverb',
    'ness': 'changes adjective to noun',
    'ment': 'changes verb to noun',
    'able': 'changes verb to adjective',
    'ible': 'changes verb to adjective',
    'ful': 'changes noun to adjective',
    'less': 'changes noun to adjective',
    'tion': 'changes verb to noun',
    'sion': 'changes verb to noun',
    'ous': 'changes noun to adjective',
    'ious': 'changes noun to adjective',
    'al': 'changes noun to adjective',
    'ize': 'changes noun/adjective to verb',
    'ise': 'changes noun/adjective to verb',
    'en': 'changes adjective to verb',
    'un': 'reverses meaning',
    're': 'adds repetition meaning',
    'dis': 'reverses or negates meaning',
    'in': 'negates meaning',
    'im': 'negates meaning',
    'ir': 'negates meaning',
    'il': 'negates meaning'
}

In [19]:
def classify_morphological_change(word):
    """
    Classify morphological changes as inflectional or derivational.
    Returns list of changes with classifications and explanations.
    """
    morphemes = extract_morphemes(word)
    changes = []
    
    # Check prefixes
    for prefix in morphemes['prefixes']:
        if prefix in derivational_morphemes:
            changes.append({
                'morpheme': prefix,
                'type': 'Derivational (Prefix)',
                'explanation': derivational_morphemes[prefix]
            })
    
    # Check suffixes
    for suffix in morphemes['suffixes']:
        if suffix in inflectional_morphemes:
            changes.append({
                'morpheme': suffix,
                'type': 'Inflectional',
                'explanation': inflectional_morphemes[suffix]
            })
        elif suffix in derivational_morphemes:
            changes.append({
                'morpheme': suffix,
                'type': 'Derivational (Suffix)',
                'explanation': derivational_morphemes[suffix]
            })
    
    return {
        'word': word,
        'changes': changes if changes else [{'morpheme': '(none)', 'type': 'N/A', 'explanation': 'No affixes detected'}]
    }

In [20]:
# Display classification for each word
classification_results = []
for word in words:
    result = classify_morphological_change(word)
    for change in result['changes']:
        classification_results.append({
            'Word': result['word'],
            'Morpheme': change['morpheme'],
            'Type': change['type'],
            'Explanation': change['explanation']
        })

classification_df = pd.DataFrame(classification_results)
classification_df

,Word,Morpheme,Type,Explanation
0,unbelievably,un,Derivational (Prefix),reverses meaning
1,unbelievably,ly,Derivational (Suffix),changes adjective to adverb
2,happiest,est,Inflectional,superlative (limited to adjectives/adverbs)
3,disagreement,dis,Derivational (Prefix),reverses or negates meaning
4,disagreement,ment,Derivational (Suffix),changes verb to noun
5,rewritten,re,Derivational (Prefix),adds repetition meaning
6,rewritten,en,Derivational (Suffix),changes adjective to verb
7,wolves,es,Inflectional,plural/3rd person singular (does not change wo...
8,singing,ing,Inflectional,present participle (limited context)
9,unfolded,un,Derivational (Prefix),reverses meaning
